## Padding video streams with `tf.data`.

In [2]:
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm

import tensorflow as tf

In [3]:
video_path = '../inputs/HorseRace/'

In [4]:
#Function for data preparation.
def feature_extraction(video_path):
    '''Function extracts video frames from a video input and returns a list 
    with images in numpy array format.'''
    
    width = 800
    height = 600
    
    frames_list = []
    
    #Read the video.
    video_reader = cv2.VideoCapture(video_path)
    
    #Obtain the fram count.
    frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    for counter in range(frame_count):
        ret, frame = video_reader.read()

        if not ret:
            print('No video frames found.')
            break

        #Resize the image.
        frame = cv2.resize(frame, (width, height))

        #Append video frames to the list.
        frames_list.append(frame)
    
    video_reader.release()
    return frames_list
        

In [5]:
def get_number_of_video_frames(video):
    cap = cv2.VideoCapture(video)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return frame_count

In [6]:
get_number_of_video_frames('../inputs/HorseRace/v_HorseRace_g04_c03.avi')

78

In [7]:
get_number_of_video_frames('../inputs/HorseRace/v_HorseRace_g06_c05.avi')

206

In [8]:
def load_video(video_dir):
    '''Function loads videos from a directory and stores them in a list.'''
    total_labels = []
    total_video_frames = []
    for file in tqdm(os.listdir(video_dir)):
        full_path = os.path.join(video_dir, file)
        
        # Extract frames from the video.
        frames_list = feature_extraction(full_path)
        frames_list = np.array(frames_list, dtype = 'float16')
        
        #Append labels.
        total_labels.append(full_path)
        total_video_frames.append(frames_list)
    
    return total_video_frames, total_labels

In [9]:
data, labels = load_video('../inputs/HorseRace/')

  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
padded_data = tf.keras.preprocessing.sequence.pad_sequences(data)

padded_data.shape

(2, 206, 600, 800, 3)

In [11]:
tf_data = tf.data.Dataset.from_tensors(padded_data)
tf_data.element_spec

2022-10-30 15:49:54.957456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-30 15:49:54.957494: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-30 15:49:54.957528: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debonair): /proc/driver/nvidia/version does not exist
2022-10-30 15:49:54.957864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-30 15:49:54.959332: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2373120000 exceeds

TensorSpec(shape=(2, 206, 600, 800, 3), dtype=tf.int32, name=None)